<a href="https://colab.research.google.com/github/budennovsk/Pandas/blob/master/pyboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install py-boost==0.5.1
# !pip install cupy-cuda110 py-boost
!pip install cupy-cuda12x py-boost


In [ ]:
!nvidia-smi

In [ ]:
import os
# os.environ['PYBOOST_GPU_DISABLE'] = '1'  # Отключаем GPU

# set the device to run
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# os.makedirs('../data', exist_ok=True)

import numpy as np
import pandas as pd
from pandas import Series, DataFrame

from sklearn.model_selection import train_test_split

from py_boost import gpu

from sklearn.datasets import make_regression


In [ ]:

# X, y = make_regression(150000, 100, n_targets=10, random_state=42)
# X_test, y_test = X[:50000], y[:50000]
# X, y = X[-50000:], y[-50000:]

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

tscv = KFold(n_splits=5,shuffle=False)



data1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Симбирсофт/Северсталь/data/data_wide_32.csv',sep=';')
data = data1[data1['Дата/время производства'] <'2025-01-01']
# X = data[[i for i in data.columns if '2000_signal' in i]]
# y = data[[i for i in data.columns if '650_signal' in i]]

mae_scores=[]
mse_scores=[]
for fold, (train_idx, test_idx) in enumerate(tscv.split(data)):
    train_data = data.iloc[train_idx]
    test_data = data.iloc[test_idx]

    model = gpu.boosting.GradientBoosting('mse', 'r2_score',
                         ntrees=1000, lr=.1, verbose=100, es=200, lambda_l2=1,
                         subsample=.4, colsample=.8, min_data_in_leaf=3, min_gain_to_split=0,
                         max_bin=30, max_depth=6)

    model.fit(train_data[[i for i in data.columns if '650_signal' in i]], train_data[[i for i in data.columns if '2000_signal' in i]],eval_sets=[{'X': test_data[[i for i in data.columns if '650_signal' in i]], 'y': test_data[[i for i in data.columns if '2000_signal' in i]]},])

    pred_values = model.predict(test_data[[i for i in data.columns if '2000_signal' in i]])

    mae = mean_absolute_error(test_data[[i for i in data.columns if '650_signal' in i]].values, pred_values,multioutput='uniform_average')
    mse = mean_squared_error(test_data[[i for i in data.columns if '650_signal' in i]].values, pred_values,multioutput='uniform_average')
    # smape = 100 * np.mean(2 * np.abs(true_values - pred_values) / (np.abs(true_values) + np.abs(pred_values) + 1e-8))
    # r2 = r2_score(true_values, pred_values)  # <-- Добавляем R2

    mae_scores.append(mae)
    mse_scores.append(mse)

print("\nСреднее MSE по фолдам:", np.mean(mse_scores))
print("Среднее MAE по фолдам:", np.mean(mae_scores))
    # print("Среднее R² по фолдам:", np.mean(r2_scores))
    # print("Среднее sMAPE по фолдам:", np.mean(smape_scores))


